# Part II: Feature extarction

### Importations

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from urllib.parse import urlparse,urlencode
import ipaddress
import re
import socket
import requests
import time

## II.1. Features based on Adress

Here are some obvious features related to URL format and other suspicious forms that can be included by attackers to propagate their threats. In this part we code some functions that calculate those attributes. 


**II.1.1 Domain of the URL**

In [32]:
def getDomain(url):  
    domain = urlparse(url).netloc
    if re.match(r"^www.",domain):
        domain = domain.replace("www.","")
        return(domain)
    return("NULL") 
# if we can't extract the domain du to anomaly in the link we return None

 **II.1.2 IP Address in the URL**

In [33]:
def getIPAddress(url):
    try:
        address=socket.gethostbyname(getDomain(url))
        return(address)
    except:
        return(None)
        

**II.1.3 "@" Symbol in URL**

In [34]:
def getExitanceSymbol(url):
    if '@' in url:
        return(1)
    else:
        return(0)

**II.1.4 Length of URL** 



In [35]:
def LengthUrl(url):
    return(len(url))

**II.1.5 Length of Domain**

In [36]:
def LengthDomain(url):
    return(len(getDomain(url)))

**II.1.6 Depth of Domain**

In [37]:
def getDepth(url):
    ch=getDomain(url)
    l=ch.split('/')
    depth=0
    for i in l:
        if len(i)>0:
            depth=depth+1
    return(depth)

**II.1.7 Number of "-" in URL**

In [38]:
def occurence_tirets(url):
    l=url.split('-')
    nbr=0
    for i in l:
        if len(i)>0:
            nbr=nbr+1
    return(nbr)

**II.1.8 Redirection "//" in URL**

In [39]:
#  redirection to other website
def getPositionRedirection(url):
    l=[i for i in url]
    for i in range (len(l)-1):
        if l[i]=="/" and l[i+1]=="/":
            return(i)
    return(None)

**II.1.9 Number of % in URL**

In [40]:
# nombre de %
def getNumberPourcentage(url):
    l=url.split('%')
    nbr=0
    for i in l:
        if len(i)>0:
            nbr=nbr+1
    return(nbr)

**II.1.10 "https" in Domain name**

In [41]:
#Detect the string https in domain

def httpInDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

**II.1.11 Using URL Shortening Services “TinyURL”**

In [42]:
#Tiny urls:
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

## II.2.HTML & JS based Features

In this section, we will extract information based on the HTML and JavaScript features of websites.

**II.2.1 IFrame** 

In [43]:
 import requests

In [44]:
def iframe(url):
    alarm = threading.Timer(delay_time, watchdog) # works only on linux , used to add timeout
    alarm.start()
    try:
        response = requests.get(url)
    except:
        response = ""
    if response == "":
        return 1
    else:
        if re.findall(r"[|]", response.text):
            return 0
        else:
            return 1
    alarm.cancel()


**II.2.2 Status Bar Customization:**

In [45]:
def mouseOver(url): 
    alarm = threading.Timer(delay_time, watchdog)
    alarm.start()
    try:
        response = requests.get(url)
    
        if response == "" :
            return 1
        else:
            if re.findall("", response.text):
              return 1
            else:
              return 0
    except:
            return(None)
    alarm.cancel()

**II.2.3 Number of Redirection:**

In [46]:
def NumberRedirects(url):
    response = requests.get(url, headers={'User-Agent': 'Google Chrome'})
    if response.history:
        return 1 + NumberRedirects(response.url)        
    return 0

## II.3. Abnormal based features

**II.3.1 URL of Anchor:**


In [47]:
def URLofAnchor(url):
    domain = urlparse(url).netloc 

    soup = BeautifulSoup(requests.get(url).content, "html.parser").find_all("a")
    total = len(soup)
    count = 0
    for ele in soup[:]:
        ele = str(ele)
        #print(ele)
        if 'href="https://' in ele and domain not in ele:
            count+=1
        if 'href="#"' in ele or 'href="#content"' in ele or 'href="#skip"' in ele or 'href="JavaScript ::void(0)"' in ele:
            count+=1

    if total == 0:
        return None
    return count/total*100

# II.4. Domain based Features

**II.4.1 Number of Months Alive:**

In [48]:
import whois
from dateutil import relativedelta

In [49]:

def checkWHOIS_exists_months(url):
    #url = re.sub(r'(?:https?://)([^/]+)(?:\S+)', r"\1", url)
    w = whois.whois(url)
    #print(w)
    if w["creation_date"] == None:
        print("test")
        return None
    else:
        date_crea = w["creation_date"]
        try:
            if len(date_crea) > 1:
                date_crea = date_crea[-1]
        except:
            pass
        diff = relativedelta.relativedelta(date.today(),date_crea)
        print(diff)
        return diff.months + diff.years * 12

**II.4.2 isinAlexa:**


In [50]:
import csv

In [51]:
def isinAlexa(url):
    #url = re.sub(r'(?:https?://)([^/]+)(?:\S+)', r"\1", url)
    url = url[12:]
    print(url)
    with open('top-1m.csv', 'rt') as f:
        reader = csv.reader(f, delimiter=',') # good point by @paco
        for row in reader:
            if row[1] == url:
                return 1
    return 0

**II.4.3 Number of Links Pointing to Page:**

In [52]:
def NumbLinkToDomain(url):
    domain = re.sub(r'(?:https?://)([^/]+)(?:\S+)', r"\1", url)
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')

    yes=0

    for link in soup.find_all('a'):
        link = link.get('href')
        if domain in link:
            yes+=1
    return yes

**II.4.4 Indexed by Google:**

In [53]:
def indexbyGoogle(url):
    google = "https://www.google.com/search?q=site:" + url + "&hl=en"
    response = requests.get(google, cookies={"CONSENT": "YES+1"})
    soup = BeautifulSoup(response.content, "html.parser")
    not_indexed = re.compile("did not match any documents")

    if soup(text=not_indexed):
        return 0
    else:
        return 1

## II.5 URL Features computation

In [54]:
def featureExtractions(url,label):

    features = []
    
    features.append(url)
    features.append(URLofAnchor(url))
    features.append(NumberRedirects(url))
    features.append(checkWHOIS_exists_months(url))
    features.append(isinAlexa(url))
    features.append(NumbLinkToDomain(url))
    features.append(indexbyGoogle(url))
    features.append(getDomain(url))
    features.append(getIPAddress(url))
    features.append(getExitanceSymbol(url))
    features.append(LengthUrl(url))
    features.append(LengthDomain(url))
    features.append(getDepth(url))
    features.append(occurence_tirets(url))
    features.append(getPositionRedirection(url))
    features.append(getNumberPourcentage(url))
    features.append(httpInDomain(url))
    features.append(tinyURL(url))
    features.append(iframe(url))
    features.append(mouseOver(url))
    features.append(label)
    return features

**II.5.1 Phishing Websites**

In [55]:
#load phishing websites
import pandas as pd
df = pd.read_csv('./Datasets/phish_file_brut.csv')
df=df['url']
df.head()

0    https://75e523b0-579a-4d05-aa9e-2d1ccde20fc0.i...
1      http://vystar.security.canarotransportes.com.br
2    http://vintedsite-market.45444342655.xyz/qiaex...
3               https://fwwwfrererre.000webhostapp.com
4            https://e-tax.form.3649-indentificamz.com
Name: url, dtype: object

In [56]:
features = []
label = 1 # label of phish
for i in range(0, df.shape[0]):
    print(i) # to track the execution
    url = df[i] 
    features.append(featureExtractions(url,label))

0
test


NameError: name 'threading' is not defined

In [25]:
feature_names = ["Link","URLofAnchor","NumberRedirects","checkWHOIS_exists","isinAlexa","NumbLinkToDomain","indexbyGoogle","Domain","Have_IP","Have_@","URL_Length","Domain_Length","URL_Depth","occurence_tirets","Redirection","Number_%","https_Domain","TinyURL","iFrame","MouseOver","label"]

#coverting the list to a dataframe
phishing = pd.DataFrame(features, columns= feature_names)
phishing.head()

,Link,Domain,Have_IP,Have_@,URL_Length,Domain_Length,URL_Depth,occurence_tirets,Redirection,Number_%,https_Domain,TinyURL,label
0,http://shadetreetechnology.com/V4/validation/a...,shadetreetechnology.com,34.102.136.180,0,77,23,1,1,5,1,0,0,1
1,https://support-appleld.com.secureupdate.duila...,support-appleld.com.secureupdate.duilawyeryork...,None,0,126,50,1,2,6,1,0,0,1
2,http://appleid.apple.com-app.es/,appleid.apple.com-app.es,None,0,32,24,1,2,5,1,0,0,1
3,http://www.shadetreetechnology.com/V4/validati...,shadetreetechnology.com,34.102.136.180,0,81,23,1,1,5,1,0,0,1
4,http://html.house/l7ceeid6.html,html.house,104.236.175.252,0,31,10,1,1,5,1,0,0,1


In [26]:
#Save the dataframe in a csv file
phishing.to_csv('./Datasets/phish_file.csv', index= False)

**II.5.2 Ligitimate websites**

In [ ]:
#load ligitimate websites
import pandas as pd
df = pd.read_csv('./Datasets/ligit_file_brut.csv')
df=df['url']
df.head()

In [ ]:
features = []
label = 0 # label of phish
for i in range(0, df.shape[0]):
    print(i) # to track the execution
    url = df[i] 
    features.append(featureExtractions(url,label))

In [ ]:
feature_names = ["Link","URLofAnchor","NumberRedirects","checkWHOIS_exists","isinAlexa","NumbLinkToDomain","indexbyGoogle","Domain","Have_IP","Have_@","URL_Length","Domain_Length","URL_Depth","occurence_tirets","Redirection","Number_%","https_Domain","TinyURL","iFrame","MouseOver","label"]


#coverting the list to a dataframe
phishing = pd.DataFrame(features, columns= feature_names)
phishing.head()

In [ ]:
#Save the dataframe in a csv file
phishing.to_csv('./Datasets/ligit_file.csv', index= False)